# Texas General Election Data Extractor (2018-2024)

This notebook extracts and processes:
- **Election Results**: Statewide and county-level results from Texas Secretary of State
- **Voter Registration Data**: Historical registration figures by county
- **Demographic Data**: Census voting and registration supplements

## Data Sources
- Texas Secretary of State: https://www.sos.state.tx.us/elections/
- U.S. Census Bureau CPS Voting Supplement
- MIT Election Data + Science Lab

## Setup and Dependencies

In [1]:
# Install required packages
!pip install pandas requests beautifulsoup4 lxml openpyxl xlrd --quiet

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os
import io
import re
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Create output directory
OUTPUT_DIR = 'texas_election_data'
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Request headers to avoid blocks
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
}

print(f"Output directory: {OUTPUT_DIR}")
print(f"Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

C:\Users\spdal\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


Output directory: texas_election_data
Timestamp: 2026-01-14 11:36:10


---
## 1. Texas Secretary of State - Election Results

Download official certified election results for general elections.

In [4]:
# Texas SOS Election Results URLs
# These are direct links to downloadable data files from the TX SOS website

TX_SOS_BASE = "https://www.sos.state.tx.us/elections/historical"

# Election result pages by year
ELECTION_PAGES = {
    2024: f"{TX_SOS_BASE}/2024.shtml",
    2022: f"{TX_SOS_BASE}/2022.shtml",
    2020: f"{TX_SOS_BASE}/2020.shtml",
    2018: f"{TX_SOS_BASE}/2018.shtml"
}

def get_election_data_links(year):
    """Scrape the TX SOS page to find links to election data files."""
    url = ELECTION_PAGES.get(year)
    if not url:
        return []
    
    try:
        response = requests.get(url, headers=HEADERS, timeout=30)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        
        links = []
        for a in soup.find_all('a', href=True):
            href = a['href']
            # Look for Excel/CSV files or canvass reports
            if any(ext in href.lower() for ext in ['.xls', '.xlsx', '.csv', 'canvass']):
                full_url = href if href.startswith('http') else f"https://www.sos.state.tx.us{href}"
                links.append({'text': a.get_text(strip=True), 'url': full_url})
        
        return links
    except Exception as e:
        print(f"Error fetching {year} page: {e}")
        return []

# Display available data links for each year
for year in [2024, 2022, 2020, 2018]:
    print(f"\n=== {year} General Election ===")
    links = get_election_data_links(year)
    if links:
        for link in links[:10]:  # Show first 10
            print(f"  - {link['text'][:60]}..." if len(link['text']) > 60 else f"  - {link['text']}")
    else:
        print("  No direct data links found (may need manual download)")


=== 2024 General Election ===
Error fetching 2024 page: 403 Client Error: Forbidden for url: https://www.sos.state.tx.us/elections/historical/2024.html
  No direct data links found (may need manual download)

=== 2022 General Election ===
Error fetching 2022 page: 403 Client Error: Forbidden for url: https://www.sos.state.tx.us/elections/historical/2022.shtml
  No direct data links found (may need manual download)

=== 2020 General Election ===
Error fetching 2020 page: 403 Client Error: Forbidden for url: https://www.sos.state.tx.us/elections/historical/2020.shtml
  No direct data links found (may need manual download)

=== 2018 General Election ===
Error fetching 2018 page: 403 Client Error: Forbidden for url: https://www.sos.state.tx.us/elections/historical/2018.shtml
  No direct data links found (may need manual download)


### 1.1 Download Statewide Canvass Reports

The canvass reports contain certified vote totals for all races.

In [5]:
# Known direct URLs for Texas canvass/results data
# These URLs may need updating as the SOS updates their site

KNOWN_DATA_URLS = {
    # Presidential/Federal races
    '2024_general': 'https://results.texas-election.com/static/exports/2024GE/county.csv',
    '2022_general': 'https://results.texas-election.com/static/exports/2022GE/county.csv',
    '2020_general': 'https://results.texas-election.com/static/exports/2020GE/county.csv',
    '2018_general': 'https://results.texas-election.com/static/exports/2018GE/county.csv',
}

def download_election_data(url, filename):
    """Download election data file and return as DataFrame."""
    try:
        response = requests.get(url, headers=HEADERS, timeout=60)
        response.raise_for_status()
        
        # Save raw file
        filepath = os.path.join(OUTPUT_DIR, filename)
        with open(filepath, 'wb') as f:
            f.write(response.content)
        
        # Try to parse as DataFrame
        if filename.endswith('.csv'):
            df = pd.read_csv(io.BytesIO(response.content))
        elif filename.endswith(('.xls', '.xlsx')):
            df = pd.read_excel(io.BytesIO(response.content))
        else:
            df = None
        
        print(f"✓ Downloaded: {filename}")
        return df, filepath
    except Exception as e:
        print(f"✗ Failed to download {filename}: {e}")
        return None, None

# Attempt downloads
election_dataframes = {}
for key, url in KNOWN_DATA_URLS.items():
    ext = url.split('.')[-1]
    filename = f"tx_{key}.{ext}"
    df, path = download_election_data(url, filename)
    if df is not None:
        election_dataframes[key] = df

✗ Failed to download tx_2024_general.csv: Error tokenizing data. C error: Expected 1 fields in line 9, saw 3

✗ Failed to download tx_2022_general.csv: Error tokenizing data. C error: Expected 1 fields in line 9, saw 3
✗ Failed to download tx_2020_general.csv: Error tokenizing data. C error: Expected 1 fields in line 9, saw 3
✗ Failed to download tx_2018_general.csv: Error tokenizing data. C error: Expected 1 fields in line 9, saw 3


### 1.2 Alternative: MIT Election Data Lab

If direct TX SOS downloads fail, use cleaned data from MIT.

In [ ]:
# MIT Election Data + Science Lab - County-level returns
MIT_URLS = {
    'president_county': 'https://dataverse.harvard.edu/api/access/datafile/:persistentId?persistentId=doi:10.7910/DVN/VOQCHQ/HEIJCQ',
    'senate_county': 'https://dataverse.harvard.edu/api/access/datafile/:persistentId?persistentId=doi:10.7910/DVN/PEJ5QU/ESFZKF',
}

def download_mit_data():
    """Download county-level election data from MIT Election Lab."""
    datasets = {}
    
    for name, url in MIT_URLS.items():
        try:
            print(f"Downloading MIT {name} data...")
            response = requests.get(url, headers=HEADERS, timeout=120)
            response.raise_for_status()
            
            # Save file
            filepath = os.path.join(OUTPUT_DIR, f"mit_{name}.csv")
            with open(filepath, 'wb') as f:
                f.write(response.content)
            
            # Parse and filter to Texas
            df = pd.read_csv(io.BytesIO(response.content))
            if 'state' in df.columns:
                df_tx = df[df['state'].str.upper() == 'TEXAS'].copy()
            elif 'state_po' in df.columns:
                df_tx = df[df['state_po'] == 'TX'].copy()
            else:
                df_tx = df
            
            datasets[name] = df_tx
            print(f"  ✓ {name}: {len(df_tx)} Texas records")
            
        except Exception as e:
            print(f"  ✗ Failed: {e}")
    
    return datasets

# Download MIT data
mit_data = download_mit_data()

---
## 2. Voter Registration Data

Historical voter registration figures from TX Secretary of State.

In [ ]:
# TX SOS Voter Registration History
VR_URL = "https://www.sos.state.tx.us/elections/historical/vrinfo.shtml"

def scrape_voter_registration_page():
    """Scrape voter registration data links from TX SOS."""
    try:
        response = requests.get(VR_URL, headers=HEADERS, timeout=30)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Find all links to Excel/PDF files
        vr_links = []
        for a in soup.find_all('a', href=True):
            href = a['href']
            text = a.get_text(strip=True)
            if any(ext in href.lower() for ext in ['.xls', '.xlsx', '.pdf']):
                full_url = href if href.startswith('http') else f"https://www.sos.state.tx.us{href}"
                vr_links.append({'text': text, 'url': full_url})
        
        return vr_links
    except Exception as e:
        print(f"Error: {e}")
        return []

vr_links = scrape_voter_registration_page()
print("Available Voter Registration Data Files:")
for link in vr_links:
    print(f"  - {link['text']}: {link['url']}")

In [ ]:
# Download voter registration Excel files
vr_dataframes = {}

for link in vr_links:
    if '.xls' in link['url'].lower():
        # Create safe filename
        safe_name = re.sub(r'[^\w\s-]', '', link['text'])[:50].strip()
        ext = 'xlsx' if '.xlsx' in link['url'] else 'xls'
        filename = f"vr_{safe_name}.{ext}"
        
        try:
            response = requests.get(link['url'], headers=HEADERS, timeout=60)
            response.raise_for_status()
            
            filepath = os.path.join(OUTPUT_DIR, filename)
            with open(filepath, 'wb') as f:
                f.write(response.content)
            
            # Try to read as DataFrame
            try:
                df = pd.read_excel(io.BytesIO(response.content))
                vr_dataframes[safe_name] = df
                print(f"✓ {filename}: {len(df)} rows")
            except:
                print(f"✓ Downloaded {filename} (could not parse as DataFrame)")
                
        except Exception as e:
            print(f"✗ Failed: {link['text']} - {e}")

---
## 3. Census Demographic Voting Data

U.S. Census Current Population Survey (CPS) Voting and Registration Supplement.
This provides demographic breakdowns of who votes by age, race, education, etc.

In [ ]:
# Census CPS Voting Supplement Data
# These are state-level tables from census.gov

CENSUS_VOTING_URLS = {
    # Table 4a - Reported Voting and Registration by Sex, Race, Hispanic Origin (State)
    '2024_race_sex': 'https://www2.census.gov/programs-surveys/cps/tables/p20/590/table04a.xlsx',
    '2022_race_sex': 'https://www2.census.gov/programs-surveys/cps/tables/p20/586/table04a.xlsx',
    '2020_race_sex': 'https://www2.census.gov/programs-surveys/cps/tables/p20/585/table04a.xlsx',
    '2018_race_sex': 'https://www2.census.gov/programs-surveys/cps/tables/p20/583/table04a.xlsx',
    
    # Table 4b - Reported Voting and Registration by Age (State)
    '2024_age': 'https://www2.census.gov/programs-surveys/cps/tables/p20/590/table04b.xlsx',
    '2022_age': 'https://www2.census.gov/programs-surveys/cps/tables/p20/586/table04b.xlsx',
    '2020_age': 'https://www2.census.gov/programs-surveys/cps/tables/p20/585/table04b.xlsx',
    '2018_age': 'https://www2.census.gov/programs-surveys/cps/tables/p20/583/table04b.xlsx',
}

def download_census_voting_data():
    """Download Census CPS voting supplement tables."""
    census_data = {}
    
    for name, url in CENSUS_VOTING_URLS.items():
        filename = f"census_{name}.xlsx"
        try:
            response = requests.get(url, headers=HEADERS, timeout=60)
            response.raise_for_status()
            
            filepath = os.path.join(OUTPUT_DIR, filename)
            with open(filepath, 'wb') as f:
                f.write(response.content)
            
            # Census tables often have header rows to skip
            try:
                df = pd.read_excel(io.BytesIO(response.content), header=[3,4])
                census_data[name] = df
                print(f"✓ {name}: {len(df)} rows")
            except:
                # Try simpler read
                df = pd.read_excel(io.BytesIO(response.content))
                census_data[name] = df
                print(f"✓ {name}: {len(df)} rows (raw)")
                
        except Exception as e:
            print(f"✗ {name}: {e}")
    
    return census_data

census_data = download_census_voting_data()

### 3.1 Filter Census Data to Texas

In [ ]:
def extract_texas_from_census(df, df_name):
    """Extract Texas rows from Census CPS data."""
    # Census tables have state names in various columns
    # Try to find and filter to Texas
    
    tx_df = None
    
    for col in df.columns:
        col_str = str(col).lower()
        if 'state' in col_str or 'unnamed' in col_str:
            # Check if this column contains state names
            col_values = df[col].astype(str).str.lower()
            if col_values.str.contains('texas').any():
                tx_mask = col_values.str.contains('texas')
                tx_df = df[tx_mask].copy()
                break
    
    if tx_df is not None and len(tx_df) > 0:
        print(f"  {df_name}: Found {len(tx_df)} Texas rows")
        return tx_df
    else:
        print(f"  {df_name}: Could not isolate Texas (returning full dataset)")
        return df

# Extract Texas data from each Census table
texas_census_data = {}
print("Extracting Texas from Census data:")
for name, df in census_data.items():
    texas_census_data[name] = extract_texas_from_census(df, name)

---
## 4. Create Combined/Cleaned Datasets

In [ ]:
# Create a summary of all downloaded data

print("="*60)
print("SUMMARY OF DOWNLOADED DATA")
print("="*60)

print("\n--- Election Results ---")
for name, df in election_dataframes.items():
    print(f"{name}: {df.shape[0]} rows x {df.shape[1]} columns")

print("\n--- MIT Election Lab Data ---")
for name, df in mit_data.items():
    print(f"{name}: {df.shape[0]} rows x {df.shape[1]} columns")
    
print("\n--- Voter Registration Data ---")
for name, df in vr_dataframes.items():
    print(f"{name}: {df.shape[0]} rows x {df.shape[1]} columns")

print("\n--- Census Demographic Data ---")
for name, df in texas_census_data.items():
    print(f"{name}: {df.shape[0]} rows x {df.shape[1]} columns")

print("\n" + "="*60)
print(f"All files saved to: {os.path.abspath(OUTPUT_DIR)}")

### 4.1 Export Clean CSVs for Modeling

In [ ]:
# Export all dataframes to CSV for easy model consumption

CLEAN_DIR = os.path.join(OUTPUT_DIR, 'clean_csvs')
os.makedirs(CLEAN_DIR, exist_ok=True)

def safe_to_csv(df, name, directory):
    """Export DataFrame to CSV with safe naming."""
    safe_name = re.sub(r'[^\w\s-]', '', name).strip().replace(' ', '_')
    filepath = os.path.join(directory, f"{safe_name}.csv")
    df.to_csv(filepath, index=False)
    return filepath

exported_files = []

# Export election results
for name, df in election_dataframes.items():
    path = safe_to_csv(df, f"election_{name}", CLEAN_DIR)
    exported_files.append(path)
    print(f"✓ {path}")

# Export MIT data
for name, df in mit_data.items():
    path = safe_to_csv(df, f"mit_{name}_texas", CLEAN_DIR)
    exported_files.append(path)
    print(f"✓ {path}")

# Export voter registration
for name, df in vr_dataframes.items():
    path = safe_to_csv(df, f"vr_{name}", CLEAN_DIR)
    exported_files.append(path)
    print(f"✓ {path}")

# Export census data
for name, df in texas_census_data.items():
    path = safe_to_csv(df, f"census_{name}_texas", CLEAN_DIR)
    exported_files.append(path)
    print(f"✓ {path}")

print(f"\n✓ Exported {len(exported_files)} clean CSV files to {CLEAN_DIR}")

---
## 5. Quick Data Exploration

In [ ]:
# Preview MIT presidential data if available
if 'president_county' in mit_data:
    df = mit_data['president_county']
    print("MIT Presidential Returns - Texas Counties")
    print(f"Years available: {sorted(df['year'].unique()) if 'year' in df.columns else 'N/A'}")
    print(f"\nColumns: {list(df.columns)}")
    print(f"\nSample data:")
    display(df.head(10))

In [ ]:
# Preview voter registration data if available
if vr_dataframes:
    first_key = list(vr_dataframes.keys())[0]
    df = vr_dataframes[first_key]
    print(f"Voter Registration Data: {first_key}")
    print(f"\nColumns: {list(df.columns)}")
    print(f"\nSample data:")
    display(df.head(10))

---
## 6. Manual Download Links

If automated downloads fail, use these links to manually download data:

In [ ]:
manual_links = """
TEXAS SECRETARY OF STATE - ELECTION RESULTS
============================================
• 2024 General: https://www.sos.state.tx.us/elections/historical/2024.shtml
• 2022 General: https://www.sos.state.tx.us/elections/historical/2022.shtml
• 2020 General: https://www.sos.state.tx.us/elections/historical/2020.shtml
• 2018 General: https://www.sos.state.tx.us/elections/historical/2018.shtml
• Historical Index: https://www.sos.state.tx.us/elections/historical/index.shtml

VOTER REGISTRATION
==================
• Registration by County: https://www.sos.state.tx.us/elections/historical/vrinfo.shtml

U.S. CENSUS - VOTING & REGISTRATION
====================================
• 2024: https://www.census.gov/data/tables/time-series/demo/voting-and-registration/p20-590.html
• 2022: https://www.census.gov/data/tables/time-series/demo/voting-and-registration/p20-586.html
• 2020: https://www.census.gov/data/tables/time-series/demo/voting-and-registration/p20-585.html
• 2018: https://www.census.gov/data/tables/time-series/demo/voting-and-registration/p20-583.html

MIT ELECTION DATA LAB
=====================
• County Presidential: https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/VOQCHQ
• County Senate: https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/PEJ5QU
• County House: https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/IG0UN2

TEXAS DEMOGRAPHIC CENTER
========================
• Population Estimates: https://demographics.texas.gov/Data/TPEPP/Estimates/
"""

print(manual_links)

---
## 7. Data Dictionary

In [ ]:
data_dictionary = """
DATA DICTIONARY
===============

ELECTION RESULTS (MIT/TX SOS)
-----------------------------
• year: Election year (2018, 2020, 2022, 2024)
• state: State name or postal code
• county_name/county: County name
• county_fips: 5-digit FIPS code
• office: Race type (President, Senate, Governor, etc.)
• candidate: Candidate name
• party: Political party
• candidatevotes: Votes received by candidate
• totalvotes: Total votes cast in race
• mode: Voting mode (TOTAL, ELECTION DAY, EARLY, MAIL, PROVISIONAL)

VOTER REGISTRATION (TX SOS)
---------------------------
• County: Texas county name
• Registration figures by date/month
• Active vs. Suspense voters

CENSUS CPS VOTING SUPPLEMENT
----------------------------
• Citizen voting-age population (CVAP)
• Registered voters
• Reported voted
• Breakdowns by:
  - Age groups (18-24, 25-44, 45-64, 65+)
  - Race/ethnicity (White, Black, Hispanic, Asian)
  - Sex (Male, Female)
  - Education level
  - Income brackets
"""

print(data_dictionary)

---
## Next Steps

1. **Verify downloads**: Check the `texas_election_data/` directory for downloaded files
2. **Clean as needed**: Some Census files may need header row adjustment
3. **Merge datasets**: Join election results with demographic data by year
4. **Feature engineering**: Create turnout rates, demographic ratios, etc.
5. **Model input**: Use `clean_csvs/` directory for model training